# Final Project

## Teammates: Prakhar Saxena, Stephen Hansen, Tharindu Mendis

In [1]:
import pandas as pd

In [2]:
dataset_dir = '../CS383_datasets/'

In [11]:
train = pd.read_csv(dataset_dir + 'train.csv')
train = train.rename(columns={'Dates' : 'DateTime', 'Descript' : 'Description'})
train['Date'] = pd.to_datetime(train['DateTime']).dt.date
train['Time'] = pd.to_datetime(train['DateTime']).dt.time
# don't need address or resolution.
train = train[['Date', 'Time', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Category']]
train.head()

,Date,Time,DayOfWeek,PdDistrict,X,Y,Category
0,2015-05-13,23:53:00,Wednesday,NORTHERN,-122.425892,37.774599,WARRANTS
1,2015-05-13,23:53:00,Wednesday,NORTHERN,-122.425892,37.774599,OTHER OFFENSES
2,2015-05-13,23:33:00,Wednesday,NORTHERN,-122.424363,37.800414,OTHER OFFENSES
3,2015-05-13,23:30:00,Wednesday,NORTHERN,-122.426995,37.800873,LARCENY/THEFT
4,2015-05-13,23:30:00,Wednesday,PARK,-122.438738,37.771541,LARCENY/THEFT


In [12]:
test = pd.read_csv(dataset_dir + 'test.csv')
test = test.rename(columns={'Dates' : 'DateTime'})
test['Date'] = pd.to_datetime(test['DateTime']).dt.date
test['Time'] = pd.to_datetime(test['DateTime']).dt.time
# don't need address or resolution.
test = test[['Date', 'Time', 'DayOfWeek', 'PdDistrict', 'X', 'Y']]
test.head()

,Date,Time,DayOfWeek,PdDistrict,X,Y
0,2015-05-10,23:59:00,Sunday,BAYVIEW,-122.399588,37.735051
1,2015-05-10,23:51:00,Sunday,BAYVIEW,-122.391523,37.732432
2,2015-05-10,23:50:00,Sunday,NORTHERN,-122.426002,37.792212
3,2015-05-10,23:45:00,Sunday,INGLESIDE,-122.437394,37.721412
4,2015-05-10,23:45:00,Sunday,INGLESIDE,-122.437394,37.721412
